# Traveling Salesman Problem
## Cargo freight scheduling

This was an assignment from my Operations Analytics course. The use case here is to devise the best route for shipping cargo across the country.

In [1]:
from gurobipy import *
import numpy as np
import pandas as pd

In [18]:
cargo_df = pd.read_csv("cargo-city-locations.csv")
cargo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   State      68 non-null     object 
 1   City       68 non-null     object 
 2   Latitude   68 non-null     float64
 3   Longitude  68 non-null     float64
dtypes: float64(2), object(2)
memory usage: 2.2+ KB


### Part 1: Building our data

**(a)** Honolulu, Hawaii and Augusta, Maine are the two cities that have the highest travel travel time, which is 9.07 hours.

**(b)** Trenton, New Jersery and Philadelphia, Pennsylvania are the two cities have the smallest travel time, which is 0.05 hours.

**(c)** Springfield, Illinois has the smallest average travel time to all of the other cities, which is 1.60 hours.

In [19]:
# Index locations
cargo_df['OriginInd'] = cargo_df.index

# Convert lat and lon from degree to radian
cargo_df['Lat_r'] = cargo_df['Latitude']/360 * 2*np.pi
cargo_df['Lon_r'] = cargo_df['Longitude']/360 * 2*np.pi

# Get location pairs
cargo_location_pairs = cargo_df.merge(cargo_df, how = "cross")
cargo_location_pairs = cargo_location_pairs.loc[cargo_location_pairs['OriginInd_x'] != cargo_location_pairs['OriginInd_y']]

In [20]:
# Rename
cargo_location_pairs.rename(columns = {'OriginInd_x': 'OriginInd', 'OriginInd_y': 'DestinationInd'}, inplace=True)
cargo_location_pairs.columns

Index(['State_x', 'City_x', 'Latitude_x', 'Longitude_x', 'OriginInd',
       'Lat_r_x', 'Lon_r_x', 'State_y', 'City_y', 'Latitude_y', 'Longitude_y',
       'DestinationInd', 'Lat_r_y', 'Lon_r_y'],
      dtype='object')

In [21]:
# Compute haversine angle, distance, and time
cargo_location_pairs['haversine'] = (1 - np.cos(cargo_location_pairs['Lat_r_y'] - cargo_location_pairs['Lat_r_x']))/2 + np.cos(cargo_location_pairs['Lat_r_x'])*np.cos(cargo_location_pairs['Lat_r_y'])*(1 - np.cos(cargo_location_pairs['Lon_r_y'] - cargo_location_pairs['Lon_r_x']))/2
cargo_location_pairs['distance'] = 2*6378.137*np.arcsin( np.sqrt(cargo_location_pairs['haversine']) )
cargo_location_pairs['travel_hours'] = cargo_location_pairs['distance'] / 908

In [22]:
# pd.options.display.max_rows = 1000

# Sanity check
cargo_location_pairs[(cargo_location_pairs['City_x'] == "Des Moines") & (cargo_location_pairs['City_y'] == "Baton Rouge")]

,State_x,City_x,Latitude_x,Longitude_x,OriginInd,Lat_r_x,Lon_r_x,State_y,City_y,Latitude_y,Longitude_y,DestinationInd,Lat_r_y,Lon_r_y,haversine,distance,travel_hours
969,Iowa,Des Moines,41.591087,-93.603729,14,0.725901,-1.633693,Louisiana,Baton Rouge,30.457069,-91.187393,17,0.531576,-1.59152,0.009698,1258.226139,1.385712


In [23]:
# Highest travel time
max_time = cargo_location_pairs['travel_hours'].max()
cargo_location_pairs[cargo_location_pairs['travel_hours'] == max_time]

,State_x,City_x,Latitude_x,Longitude_x,OriginInd,Lat_r_x,Lon_r_x,State_y,City_y,Latitude_y,Longitude_y,DestinationInd,Lat_r_y,Lon_r_y,haversine,distance,travel_hours
562,Hawaii,Honolulu,21.307442,-157.857376,8,0.371885,-2.755131,Maine,Augusta,44.307167,-69.781693,18,0.773306,-1.217920,0.361898,8233.880502,9.06815
1232,Maine,Augusta,44.307167,-69.781693,18,0.773306,-1.217920,Hawaii,Honolulu,21.307442,-157.857376,8,0.371885,-2.755131,0.361898,8233.880502,9.06815


In [24]:
# Lowest travel time
min_time = cargo_location_pairs['travel_hours'].min()
cargo_location_pairs[cargo_location_pairs['travel_hours'] == min_time]

,State_x,City_x,Latitude_x,Longitude_x,OriginInd,Lat_r_x,Lon_r_x,State_y,City_y,Latitude_y,Longitude_y,DestinationInd,Lat_r_y,Lon_r_y,haversine,distance,travel_hours
2026,New Jersey,Trenton,40.220596,-74.769913,29,0.701982,-1.304981,Pennsylvania,Philadelphia,39.952600,-75.165200,54,0.697304,-1.311880,0.000012,44.982011,0.04954
3701,Pennsylvania,Philadelphia,39.952600,-75.165200,54,0.697304,-1.311880,New Jersey,Trenton,40.220596,-74.769913,29,0.701982,-1.304981,0.000012,44.982011,0.04954


In [25]:
# Smallest average travel time
average_travel_time = pd.DataFrame(cargo_location_pairs.groupby(['State_x', 'City_x'])['travel_hours'].mean())
average_travel_time[average_travel_time['travel_hours'] ==average_travel_time['travel_hours'].min()]

,,travel_hours
State_x,City_x,
Illinois,Springfield,1.599021


### Part 2: Finding a schedule

**(a)** The average of the total travel times of the 100 randomly generated sequences, in hours, is **149.46**.

**(b)** The total travel time of the heuristic sequence, in hours, is **35.85**.

**(c)** The total travel time of the optimized sequence, in hours, is **30.55**.

In [26]:
# Create pairs of the origin and destination
origin_dest_zipped = list(zip(cargo_location_pairs['OriginInd'], cargo_location_pairs['DestinationInd']))

# Map origin-destination pair with travel time
travel_time_dict = dict( zip(origin_dest_zipped, cargo_location_pairs['travel_hours']) )

In [27]:
# Random route approach

np.random.seed(50)
nCities = 68
nSequences = 100
travel_time_list = []

for s in range(nSequences):
    temp = np.random.permutation(nCities)
    # print(f'Seq {s}: {temp}')
    # print()

    total_travel_time = 0
    for i in range(nCities-1):
        origin = temp[i]
        destination = temp[i+1]
        od_pair = (origin, destination)
        total_travel_time += travel_time_dict[od_pair]

    origin = temp[nCities-1]
    destination = temp[0]
    od_pair = (origin, destination)
    total_travel_time += travel_time_dict[od_pair]

    travel_time_list.append(total_travel_time)
    
print("The average of the total travel times of these 100 randomly generated sequences, in hours:", np.mean(travel_time_list))

The average of the total travel times of these 100 randomly generated sequences, in hours: 149.4632186659759


In [28]:
# Heuristic approach

unvisited = list(range(nCities))
origin = 51  # starts in Los Angeles
unvisited.remove(origin)

tour = []
tour.append(origin)

total_travel_time = 0

while len(unvisited) > 0:
    min_time = 100  # initiate minimum time
    
    for i in unvisited: 
        
        if i in tour:
            continue
            
        od_pair = (origin, i)
        time = travel_time_dict[od_pair]
        # print(f'{od_pair} time is {time}')
        if time < min_time:
            min_time = time
            new_origin = i
            
    origin = new_origin  # update origin
    unvisited.remove(origin)  # update unvisited list
    tour.append(origin)  # update the next element in tour
    total_travel_time += min_time  # update total travel time

# Add the trip from last city to LA
origin = tour[-1]
destination = 51
od_pair = (origin, destination)
total_travel_time += travel_time_dict[od_pair]
tour.append(destination)

In [29]:
print(tour)
print(len(tour))

[51, 56, 2, 65, 30, 5, 49, 40, 32, 22, 48, 52, 13, 16, 34, 47, 62, 39, 31, 45, 64, 19, 7, 54, 29, 50, 6, 38, 20, 28, 44, 33, 18, 37, 66, 21, 12, 24, 15, 26, 14, 35, 60, 57, 42, 55, 53, 17, 23, 3, 41, 10, 0, 9, 59, 43, 11, 25, 63, 46, 67, 36, 27, 4, 61, 58, 1, 8, 51]
69


In [30]:
print(total_travel_time)

35.851706754982374


In [31]:
# TSP approach

def getSubtours(sequence):
    subtour_list = []
    unvisited = list(range(nCities))  # track the locations we have not visited in the algorithm
    
    while ( len(unvisited) > 0 ):
        node = unvisited.pop()  # node is set to be the last element in the list, and then change the unvisited list
        
        subtour = []
        subtour.append(node)
        
        # Check if the first element of the tuple (or edge) is the current node
        # Then get the end node of this tuple (or edge)
        # filter( lambda ...) returns a list that starts at current node
        next_node = list(filter(lambda t: t[0] == node, sequence))[0][1]
        
        # This loop terminates when the node is already visited
        while (next_node in unvisited):
            subtour.append(next_node)
            unvisited.remove(next_node)
            next_node = list(filter(lambda t: t[0] == next_node, sequence))[0][1]
            
        subtour_list.append(subtour)
    
    return subtour_list


def eliminateSubtours(model, where):
    if (where == GRB.Callback.MIPSOL):
        x_val = model.cbGetSolution(x)
        sequence = [ (i,j) for (i,j) in od_pairs if x_val[i,j] > 0.5]
        subtour_list = getSubtours(sequence)
        if (len(subtour_list) > 1):
            for subtour in subtour_list:
                model.cbLazy( sum(x[i,j] for i in subtour for j in subtour if i != j) <= len(subtour) - 1)

In [32]:
# Get the set of OD pairs
od_pairs = travel_time_dict.keys()
# print(len(od_pairs))

In [33]:
# Set up model
m = Model()

x = m.addVars(od_pairs, vtype = GRB.BINARY)

for i in range(nCities):
    m.addConstr( sum(x[i,j] for j in range(nCities) if j != i ) == 1)
    m.addConstr( sum(x[j,i] for j in range(nCities) if j != i ) == 1)

m.setObjective(sum( travel_time_dict[i,j] * x[i,j] for (i,j) in od_pairs ), GRB.MINIMIZE)

m.update()

# Enable lazy constraints
m.params.LazyConstraints = 1

# Supply the callback to Gurobi:
m.optimize(eliminateSubtours)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-05
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 136 rows, 4556 columns and 9112 nonzeros
Model fingerprint: 0x387b76b5
Variable types: 0 continuous, 4556 integer (4556 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.02s
Presolved: 136 rows, 4556 columns, 9112 nonzeros
Variable types: 0 continuous, 4556 integer (4556 binary)

Root relaxation: objective 2.729102e+01, 161 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Ti

In [34]:
sequence = [ (i,j) for (i,j) in od_pairs if x[i,j].x > 0.5]
subtour_list = getSubtours(sequence)

# find the index of 51, then append lists

complete_tour = subtour_list[0]
complete_tour.append( complete_tour[0] )
print("Complete tour: ", complete_tour)

Complete tour:  [67, 36, 11, 25, 43, 5, 49, 40, 32, 22, 48, 52, 21, 66, 34, 47, 37, 33, 44, 18, 28, 20, 38, 6, 50, 29, 54, 7, 19, 64, 45, 31, 62, 39, 59, 9, 0, 10, 41, 16, 13, 12, 14, 26, 15, 24, 3, 23, 17, 53, 55, 42, 60, 57, 35, 30, 65, 2, 56, 51, 27, 4, 58, 61, 8, 1, 63, 46, 67]


In [35]:
import folium

# Figure out where the "center" of the plot should be
avg_lat = cargo_df['Latitude'].mean()
avg_lon = cargo_df['Longitude'].mean()

# Create the folium map
ussmap = folium.Map(location=[avg_lat, avg_lon], 
                  zoom_start = 4, detect_retina = True)

# Add city markers
for i in range(cargo_df.shape[0]):
    folium.CircleMarker([cargo_df['Latitude'][i], cargo_df['Longitude'][i]], 
                        popup= 'City ' + str(i) + ' -- ' + cargo_df['City'][i],
                        radius = 5,
                        color = 'blue',
                        fill = False,
                        fillcolor = 'blue').add_to(ussmap)
    
# Add tour
tour_lat_lon = list(zip( cargo_df['Latitude'][complete_tour], cargo_df['Longitude'][complete_tour])) 

poly = folium.PolyLine( tour_lat_lon, color="red", weight=2.5, opacity=1)

poly.add_to(ussmap)

ussmap